## 机器学习的四个分支

二分类问题、多分类问题和标量回归问题。这三者都是监督学习（supervised learning）的例子，其目标是学习训练输入与训练目标之间的关系。

### 监督学习

给定一组样本（通常由人工标注），它可以学会将输入数据映射到已知目标［也叫标注（annotation）］。
监督学习的分类：
- 分类：对数据进行分类
- 回归：对数据进行拟合
- 序列生成：给定一张图像，预测描述图像的文字。
- 语法树预测：给定一个句子，预测其分解生成的语法树
- 目标检测：给定一张图像，在图中特定目标的周围画一个边界框。
- 图像分割：给定一张图像，在特定物体上画一个像素级的掩模（mask）。

### 无监督学习
无监督学习是指在没有目标的情况下寻找输入数据的有趣变换，其目的在于数据可视化、数据压缩、数据去噪或更好地理解数据中的相关性。  
具体的方法：
- 降维
- 聚类


### 自监督学习

自监督学习是没有人工标注的标签的监督学习，可以将它看作没有人工参与的监督学习。  
主要的例子：
- 自编码器：其生成的目标就是未经修改的输入
- 时序监督学习：用未来的输入数据作为监督，，给定视频中过去的帧来预测下一帧，或者给定文本中前面的词来预测下一个词

### 强化学习
在强化学习中，智能体（agent）接收有关其环境的信息，并学会选择使某种奖励最大化的行动。
强化学习主要集中在研究领域，除游戏外还没有取得实践上的重大成功。


## 评估机器学习模型

将数据划分为训练集、验证集和测试集，机器学习的目的是得到可以泛化（generalize）的模型，即在前所未见的数据上表现很好的模型，而过拟合则是核心难点。

### 三种数据集
- 训练集：训练模型用的数据
- 验证集：调节模型参数的时候用的数据
- 测试集： 最后测试模型的时候用的数据

#### 验证集的作用

由于开发模型时总是需要调节模型配置，比如选择层数或每层大小［这叫作模型的超参数（hyperparameter），以便与模型参数（即权重）区分开］。这个调节过程需要使用模型在验证数据上的性能作为反馈信号。这个调节过程本质上就是一种学习。

**信息泄露**  
每次基于模型在验证集上的性能来调节模型超参数，都会有一些关于验证数据的信息泄露到模型中。

#### 验证集的操作

**简单的留出验证**  

从数据挑选一部分出来，用作验证集。如下图：
![简单的留出验证](imgs/01.jpg)

coding 如下：



In [2]:
import numpy as np
from keras import models
num_validation_samples = 10000

np.random.shuffle(data) # 打乱数据

validation_data = data[:num_validation_samples]
data = data[num_validation_samples:]

training_data = data[:]

model = get_model()
model.train(training_data)
validation_score = model.evaluate(validation_data) #评估数据的模型

# 确认好模型的超参数后，可以再次调整训练数据模型

model = get_model()
model.train(np.concatenate([training_data,validation_data]))
test_score = model.evaluate(test_data)


**K 折验证**  

K 折验证（K-fold validation）将数据划分为大小相同的 K 个分区。这个对于数据量较少的模型验证才可以。
对于每个分区 i，在剩余的 K-1 个分区上训练模型，然后在分区 i 上评估模型。最终分数等于 K 个分数的平均值。

![K折验证](imgs/02.jpg)


In [ ]:
import numpy as np

# 设定K为4，数据折成4段，也需要循环4次，
k = 4

num_val_samples = len(train_data) // k
num_epochs = 100
all_scores = []

for i in range(k):
    print('Processing fold #',i)
    val_data = train_data[i * num_val_samples 
                        : (i+1) * num_val_samples]
    val_targets = train_targets[i * num_val_samples 
                        : (i+1) * num_val_samples]
    #数据合成
    partial_train_data = np.concatenate(
        [train_data[:i * num_val_samples],
        train_data[(i+1) * num_val_samples:]],
        axis = 0)
    
    partial_train_targets = np.concatenate(
        [train_targets[:i*num_val_samples],
        train_targets[(i+1)*num_val_samples:]],
        axis = 0)
    #创建模型
    model = build_model()
    #开始训练
    model.fit(partial_train_data,
             partial_train_targets,
             epochs = num_epochs,
             batch_size = 1,
             verbose = 0)
    #进行验证
    val_mse,val_mae = model.evaluate(val_data,
                                    val_targets,
                                    verbose=0)
    all_scores.append(val_mae)

**带有打乱数据的重复 K 折验证**
具体做法是多次使用 K 折验证，在每次将数据划分为 K 个分区之前都先将数据打乱。最终分数是每次 K 折验证分数的平均值。这种方法一共要训练和评估 P×K 个模型（P是重复次数），计算代价很大。

### 评估模型的注意事项
-  数据代表性:训练集和测试集都能够代表当前数据。在将数据划分为训练集和测试集之前，通常应该随机打乱数据。
-  时间箭头:你应该始终确保测试集中所有数据的时间都晚于训练集数据。
- 数据冗余:么打乱数据并划分成训练集和验证集会导致训练集和验证集之间的数据冗余。！一定要确保训练集和验证集之间没有交集。